In [1]:
#Import the GST module -- you probably want this at the beginning of every notebook
import GST

## Creating Gatesets

In [2]:
#There are more or less three ways to create GateSet objects in GST.
# 1) By creating an empty GateSet and populating with calls to buildGate and buildVector
# 2) By a single call to buildGateset, which automates 1)
# 3) By loading from a text-format gateset file

# We'll do each of these in turn

In [3]:
#1) Create an empty gateset and build gates, rho-vectors, and E-vectors manually
stateSpace = [2] #density matrix is a 2x2 matrix
spaceLabels = [('Q0',)] #interpret the 2x2 density matrix as a single qubit named 'Q0'
gateset1 = GST.GateSet()
gateset1.set_rhoVec( GST.buildVector(stateSpace,spaceLabels,"0") )
gateset1.set_EVec(   GST.buildVector(stateSpace,spaceLabels,"1") )
gateset1.set_gate('Gi', GST.buildGate(stateSpace,spaceLabels,"I(Q0)"))
gateset1.set_gate('Gx', GST.buildGate(stateSpace,spaceLabels,"X(pi/2,Q0)"))
gateset1.set_gate('Gy', GST.buildGate(stateSpace,spaceLabels,"Y(pi/2,Q0)"))
gateset1.set_identityVec( GST.buildIdentityVector(stateSpace) )
gateset1.add_SPAM_label(0,0,'plus')
gateset1.add_SPAM_label(0,-1,'minus')


In [4]:
#2) Create a gateset in a single call to buildGateset
gateset2 = GST.buildGateset( [2], [('Q0',)],['Gi','Gx','Gy'], 
                             [ "I(Q0)","X(pi/2,Q0)", "Y(pi/2,Q0)"],
                             rhoExpressions=["0"], EExpressions=["1"], 
                             spamLabelDict={'plus': (0,0), 'minus': (0,-1) }) #basis="gm" )

In [5]:
#3) Write a text-format gateset file and read it in.
gateset3_txt = \
"""
# Example text file describing a gateset

# State prepared, specified as a state in the Pauli basis (I,X,Y,Z)
rho
PauliVec
1/sqrt(2) 0 0 1/sqrt(2)

# State measured as yes outcome, also specified as a state in the Pauli basis
E
PauliVec
1/sqrt(2) 0 0 -1/sqrt(2)

Gi
PauliMx
1 0 0 0
0 1 0 0
0 0 1 0
0 0 0 1

Gx
PauliMx
1 0 0 0
0 1 0 0
0 0 0 1
0 0 -1 0

Gy
PauliMx
1 0 0 0
0 0 0 -1
0 0 1 0
0 1 0 0

IDENTITYVEC sqrt(2) 0 0 0
SPAMLABEL plus = rho E
SPAMLABEL minus = rho remainder
"""
open("tutorial_files/Example_Gateset.txt","w").write(gateset3_txt)

gateset3 = GST.loadGateset("tutorial_files/Example_Gateset.txt")

In [6]:
#All three of the above gatesets are identical.  See for yourself by printing each one:
print "Gateset 1:\n",gateset1
print "Gateset 2:\n",gateset2
print "Gateset 3:\n",gateset3

Gateset 1:
rhoVec[0] =    0.7071        0        0   0.7071


EVec[0] =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0   1.0000        0
        0        0        0   1.0000


Gx = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0        0  -1.0000
        0        0   1.0000        0


Gy = 
   1.0000        0        0        0
        0        0        0   1.0000
        0        0   1.0000        0
        0  -1.0000        0        0



Gateset 2:
rhoVec[0] =    0.7071        0        0   0.7071


EVec[0] =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0   1.0000        0
        0        0        0   1.0000


Gx = 
   1.0000        0        0        0
        0   1.0000        0        0
        0        0        0  -1.0000
        0        0   1.0000        0

## Basic Operations with Gatesets

In [7]:
#Once you have a gateset, you can use the functions in GST.GateSetTools to modify them:
depol_gateset3 = GST.GateSetTools.depolarizeGateset(gateset3,noise=0.1) # 10% depolarization

In [8]:
#Writing a gateset as a text file
GST.writeGateset(depol_gateset3, "tutorial_files/Example_depolarizedGateset.txt", title="My Gateset")

In [9]:
#Printing more detailed information about a gateset
GST.printGatesetInfo(depol_gateset3)

rhoVec[0] =    0.7071        0        0   0.7071


EVec[0] =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0   0.9000        0
        0        0        0   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0        0   0.9000
        0        0  -0.9000        0


Gy = 
   1.0000        0        0        0
        0        0        0  -0.9000
        0        0   0.9000        0
        0   0.9000        0        0





Choi Matrices:
Choi(Gi) in pauli basis = 
   0.9250       +0j        0       +0j        0       +0j        0       +0j
        0       +0j   0.0250       +0j        0       +0j        0       +0j
        0       +0j        0       +0j   0.0250       +0j        0       +0j
        0       +0j        0       +0j        0       +0j   0.0250       +0j

  --eigenvals =  [0.024999999999999977, 0.024999999999999998, 0.02499999999999

In [10]:
import numpy as np
def SPAM( G ):
    return np.dot( G.rhoVecs[0].T, G.EVecs[0])
dp3 = depol_gateset3.copy()
#dp3.set_EVec( dp3.get_rhoVec(0), 0)
dp4 = GST.GateSetTools.depolarizeSPAM(dp3, noise=0.1)
SPAM(dp4)

array([[ 0.095]])

In [11]:
print depol_gateset3


rhoVec[0] =    0.7071        0        0   0.7071


EVec[0] =    0.7071        0        0  -0.7071


Gi = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0   0.9000        0
        0        0        0   0.9000


Gx = 
   1.0000        0        0        0
        0   0.9000        0        0
        0        0        0   0.9000
        0        0  -0.9000        0


Gy = 
   1.0000        0        0        0
        0        0        0  -0.9000
        0        0   0.9000        0
        0   0.9000        0        0



